## Goal: understanding contributors

The term "community" in this context refers to the group of people contributing to Mozilla projects. Thus, this goal could be summarized as characterizing Mozilla community based on their contributors. A contributor will be understood as a person who performs an action that can be tracked in the set of considered data sources. For example: sending a commit, opening or closing a ticket. As they will be different depending on the data source, particular actions used in each analysis will be detailed within particular goals.

The main objective of this goal is to determine a set of characteristics of contributors:

  * Projects: to which projects they contribute.
  * Organizations: to which organizations they are affiliated
  * Gender: which one is their gender
  * Age: which one is their "age" in the project (time contributing)
  * Geographical origin: where do they come from

Those goals can be refined in the following questions:

**Questions**:

* Which projects can be identified?
* Which contributors have activity related to each project?
* Which organizations can be identified?
* Which contributors are affiliated to each organization?
* Which of those contributors are hired by Mozilla, and which are not?
* Which gender are contributors?
* How long have been contributors contributing?
* Where do contributors come from?

These questions can be answered with the following metrics/data:

**Metrics**:

* List of projects
* Contributors by project
* Number of contributors by project over time
* List of organizations
* Contributors by organization
* Number of contributors by organization over time
* Contributors by groups: hired by Mozilla, the rest
* Contributors by gender
* Number of contributors by gender over time
* Time of first and last commit for each contributor
* Length of period of activity for each contributor
* Contributors by time zone (when possible)
* Contributors by city name (when possible)

All the characeterizations of developers (by project, by organization, by hired by Mozilla/rest, by gender, by period of activity, by time zone, by city name) can be a discriminator / grouping factor for the metrics defined for the next goals. Most of these metrics can be made particular for each of the considered data sources.

### Metric Calculations
First we need to load a connection against the proper ES instance. We use an external module to load credentials from a file that will not be shared. If you want to run this, please use your own credentials, just put them in a file named '.settings' (in the same directory as this notebook) following the example file 'settings.sample'.



In [14]:
import pandas

import plotly as plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go

import util as ut

from util import ESConnection
from elasticsearch_dsl import Search


es_conn = ESConnection()

In [15]:
def create_search(source):
    s = Search(using=es_conn, index=source)
    # TODO: Add bot and merges filtering.
    #s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
    return s

In [16]:
def print_result(result):
    """In case you need to check query response, call this function
    """
    print(result.to_dict()['aggregations'])

In [17]:
def print_df(result, group_field, value_field, group_column, value_column):
    df = pandas.DataFrame()

    df = df.from_dict(result.to_dict()['aggregations'][group_field]['buckets'])
    df = df.drop('doc_count', axis=1)
    df[value_field] = df[value_field].apply(lambda row: row['value'])
    df=df[['key', value_field]]
    df.columns = [group_column, value_column]

    return df

In [18]:
def stack_by_time(result, group_column, time_column, value_column, group_field, time_field, value_field):
    """Creates a dataframe based on group and time values
    """
    df = pandas.DataFrame(columns=[group_column, time_column, value_column])

    for b in result.to_dict()['aggregations'][group_field]['buckets']:
        for i in b[time_field]['buckets']:
            df.loc[len(df)] = [b['key'], i['key_as_string'], i[value_field]['value']]
    
    return df

def stack_by(result, group_column, subgroup_column, value_column, group_field, subgroup_field, value_field):
    """Creates a dataframe based on group and subgroup values
    """
    df = pandas.DataFrame(columns=[group_column, subgroup_column, value_column])

    for b in result.to_dict()['aggregations'][group_field]['buckets']:
        for i in b[subgroup_field]['buckets']:
            df.loc[len(df)] = [b['key'], i['key'], i[value_field]['value']]
    
    return df

In [19]:
def stack_by_cusum(result, group_column, time_column, value_column, group_field, time_field, value_field,\
                   staff_org_names, staff_org):
    authors_org_df = pandas.DataFrame(columns=[group_column, time_column, value_column])

    for b in result.to_dict()['aggregations'][group_field]['buckets']:
        key = b['key']
        if key in staff_org_names:
            key = staff_org
        else:    
            key  = 'Non-Employees'    

        print(b['key'], '->' ,key)

        for i in b[time_field]['buckets']:

            time = i['key_as_string']
            contributors = i[value_field]['value']

            if key in authors_org_df[group_column].unique() \
                and time in authors_org_df[authors_org_df[group_column] == key][time_column].tolist():

                authors_org_df.loc[(authors_org_df[group_column] == key) \
                                     & (authors_org_df[time_column] == time),\
                                   value_column] += contributors
                #print('1', key,  time, contributors)

            else:
                authors_org_df.loc[len(authors_org_df)] = [key, time, contributors]
                #print('2', key,  time, contributors)
    
    return authors_org_df


In [20]:
def print_stacked_bar(df, time_column, value_column, group_column):
    """Print stacked bar chart from dataframe based on time_field,
    grouped by group field.
    """
    plotly.offline.init_notebook_mode(connected=True)

    bars = []
    for group in df[group_column].unique():
        group_slice_df = df.loc[df[group_column] == group]
        bars.append(go.Bar(
            x=group_slice_df[time_column].tolist(),
            y=group_slice_df[value_column].tolist(),
            name=group))

    layout = go.Layout(
        barmode='stack'
    )

    fig = go.Figure(data=bars, layout=layout)
    plotly.offline.iplot(fig, filename='stacked-bar')

In [21]:
def add_general_date_filters(s):
    # 01/01/1998
    initial_ts = '883609200000'
    return s.filter('range', grimoire_creation_date={'gt': initial_ts})

def add_bot_filter(s):
    return s.filter('term', author_bot='false')

def add_merges_filter(s):
    return s.filter('range', files={'gt': 0})

# Let's load projects from the REVIEWED SPREADSHEET
projects = ut.read_projects("data/Contributors and Communities Analysis - Project grouping.xlsx")

initial_date = '2010-01-01'

# Answers

### List of Projects

To get the list of projects we will query ES to retrieve the unique count of commits for each project. To do that, we bucketize data based on 'project' field (to a maximum of 100 projects, given by 'size' parameter set below).

#### List of Projects: Data from Dashboard

This first table shows data directly from Mozilla's dashboard. It was not removed to offer a quick comparison against new grouping (shown below this one). 

In [36]:
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('projects', 'terms', field='project', size=100)\
    .metric('commits', 'cardinality', field='hash', precision_threshold=100000)
result = s.execute()

In [37]:
print_df(result=result, group_field='projects', value_field='commits', \
         group_column='Project', value_column='# Commits')

,Project,# Commits
0,mozilla,1710816
1,mozilla-services,137573
2,rust-lang,66434
3,servo,64383
4,mdn,12256
5,moztw,7729
6,mozilla-mobile,8325
7,aframevr,5637
8,mozmar,5388
9,mozilla-japan,4794


#### List of Projects: Data from new grouping

Following table shows projects based on the spreadsheet with the new grouping.

In [128]:
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('repos', 'terms', field='repo_name', size=100000)\
    .bucket('organizations', 'terms', field='author_org_name', size=100)\
    .metric('commits', 'cardinality', field='hash', precision_threshold=100000)
result = s.execute()
# Merge projects and repos DFs
#repos = print_df(result=result, group_field='repos', value_field='commits', \
#         group_column='Repo', value_column='# Commits')
repos = stack_by(result=result, group_column='Repo', subgroup_column='Org', value_column='# Commits',
                 group_field='repos', subgroup_field='organizations', value_field='commits')

merged_df = repos.merge(projects['Github'], on='Repo', how='left')

# Group By project
projects_df = merged_df.groupby(['Project', 'Org']).agg({'# Commits': 'sum', 'Repo': 'count'})
#projects_df = projects_df.reset_index().set_index(keys=['# Commits', 'Project', 'Org'])
#projects_df = projects_df.sort_index()
projects_df = projects_df.sort_values(by='# Commits', ascending=0)

# Print a table with this data
plotly.offline.init_notebook_mode(connected=True)
table = ff.create_table(projects_df.reset_index())
plotly.offline.iplot(table, filename='github-projects-table.html')


### Authors by Project

#### Git

Same as above, we first show data from Dashboard and then a new table using the grouping spreadsheet provided by Mozilla.

In [101]:
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('projects', 'terms', field='project', size=100)\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)
result = s.execute()

In [102]:
print_df(result=result, group_field='projects', value_field='contributors', \
         group_column='Project', value_column='# Authors')

,Project,# Authors
0,mozilla,10990
1,mozilla-services,2172
2,rust-lang,2428
3,servo,1408
4,mdn,339
5,moztw,92
6,mozilla-mobile,110
7,aframevr,221
8,mozmar,70
9,mozilla-japan,58


In [50]:
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('repos', 'terms', field='repo_name', size=100000)\
    .bucket('organizations', 'terms', field='author_org_name', size=100)\
    .bucket('contributors', 'terms', field='author_uuid', size=1000000)
result = s.execute()

# Process results to build a DataFrame
i = 0
repos_df = pandas.DataFrame(columns=['Repo', 'Org', 'uuid'])
for repo in result.to_dict()['aggregations']['repos']['buckets']:
    for org in repo['organizations']['buckets']:
        for author in org['contributors']['buckets']:
            repos_df.loc[len(repos_df)] = [repo['key'], org['key'], author['key']]
            i += 1
            if i % 10000 == 0:
                print(i, end=', ')

merged_df = repos_df.merge(projects['Github'], on='Repo', how='left')

# Group By project
projects_df = merged_df.groupby(['Project', 'Org']).agg({'uuid': pandas.Series.nunique, 
                                                         'Repo': pandas.Series.nunique})
projects_df = projects_df.sort_values(by='uuid', ascending=0)

# Print a table with this data
plotly.offline.init_notebook_mode(connected=True)
table = ff.create_table(projects_df.reset_index())
plotly.offline.iplot(table, filename='git-projects-contributors-table.html')

10000, 20000, 30000, 40000, 50000, 60000, 

**Table above: Git Authors by Projects using Spreadsheet Data**

#### Bugzilla

In this case, we didn't have any project information in Bugzilla's index, so interesting data come within the second table, built using the new grouping. We use Product and Component to assign project name to Bugzilla entries.

In [105]:
s = create_search(source='bugzilla')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('projects', 'terms', field='project', size=100)\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)
result = s.execute()

print_df(result=result, group_field='projects', value_field='contributors', \
         group_column='Project', value_column='# Authors')

,Project,# Authors
0,unknown,184708


In [11]:
######
# First retrieve Mozilla employees (triying to get everything together results in 502 error)
# so let's take partial results and merge them
######
s = create_search(source='bugzilla')

s = s.filter('terms', author_org_name=['Mozilla Staff'])

# Unique count of Authors by Project
s.aggs.bucket('product', 'terms', field='product', size=100000)\
    .bucket('component', 'terms', field='component', size=100000)\
    .bucket('contributors', 'terms', field='author_uuid', size=100000)
result = s.execute()

print('q1')

# Process results to build a DataFrame
i = 0
moz_df = pandas.DataFrame(columns=['Product', 'Component', 'uuid'])
for product in result.to_dict()['aggregations']['product']['buckets']:
    for component in product['component']['buckets']:
        for author in component['contributors']['buckets']:
            moz_df.loc[len(moz_df)] = [product['key'], component['key'], author['key']]
            i += 1
            if i % 10000 == 0:
                print(i, end=', ')

print('Moz: ', len(moz_df))
            
# Merge projects by product & components DFs
moz_merged_df = moz_df.merge(projects['Bugzilla'], on=['Product', 'Component'], how='left')
moz_merged_df['Org'] = 'Employees'

print('Moz merged: ', len(moz_merged_df))


####
# Second get results for non-employees
###
s = create_search(source='bugzilla')

s = s.exclude('terms', author_org_name=['Mozilla Staff'])

# Unique count of Authors by Project 
s.aggs.bucket('product', 'terms', field='product', size=100000)\
    .bucket('component', 'terms', field='component', size=100000)\
    .bucket('contributors', 'terms', field='author_uuid', size=100000)
result = s.execute()

print('q2')

# Process results to build a DataFrame
non_moz_df = pandas.DataFrame(columns=['Product', 'Component', 'uuid'])
i = 0
for product in result.to_dict()['aggregations']['product']['buckets']:
    for component in product['component']['buckets']:
        for author in component['contributors']['buckets']:
            non_moz_df.loc[len(non_moz_df)] = [product['key'], component['key'], author['key']]
            i += 1
            if i % 10000 == 0:
                print(i, end=', ')

print('Non-moz: ', len(non_moz_df))
            
# Merge projects by product & components DFs
non_moz_merged_df = non_moz_df.merge(projects['Bugzilla'], on=['Product', 'Component'], how='left')
non_moz_merged_df['Org'] = 'Non-Employees'

print('Non-moz merged: ', len(non_moz_merged_df))

###
# Concat both data frames into a single one
###
merged_df = pandas.concat([moz_merged_df, non_moz_merged_df])

print('After concat')


# Group By project
projects_df = merged_df.groupby(['Project', 'Org']).agg({'uuid': pandas.Series.nunique,
                                                         'Product': pandas.Series.nunique,
                                                         'Component': pandas.Series.nunique})
projects_df = projects_df.sort_values(by='uuid', ascending=0)

# Print a table with this data
plotly.offline.init_notebook_mode(connected=True)
table = ff.create_table(projects_df.reset_index())
plotly.offline.iplot(table, filename='bugzilla-projects-contributors-table.html')

q1
10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, Moz:  86562
Moz merged:  86562
q2
10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000, 170000, 180000, 190000, 200000, 210000, 220000, 230000, 240000, 250000, 260000, 270000, 280000, 290000, 300000, 310000, 320000, 330000, 340000, 350000, 360000, Non-moz:  365549
Non-moz merged:  365549
After concat


**Table above: Bugzilla Authors by Project using Spreadsheet Data**

In [48]:
merged_df.loc[(merged_df['Project'] == 'Firefox') & (merged_df['Org'] == 'Non-Employees')].groupby('Project')\
                .agg({'uuid': pandas.Series.nunique,
                      'Product': pandas.Series.nunique,
                      'Component': pandas.Series.nunique})
    
print(len(merged_df.loc[(merged_df['Project'] == 'Firefox') & (merged_df['Org'] == 'Non-Employees')]['uuid'].unique()))
print(len(merged_df['uuid'].unique()))
print(len(moz_df['uuid'].unique()))
print(len(non_moz_df['uuid'].unique()))
print(len(moz_df['uuid'].unique()) + len(non_moz_df['uuid'].unique()))

print(len(moz_merged_df['uuid'].unique()))
print(len(non_moz_merged_df['uuid'].unique()))
print(len(moz_merged_df['uuid'].unique()) + len(non_moz_merged_df['uuid'].unique()))

71348
184333
2784
181551
184335
2784
181551
184335


### Number of Authors by project over time
#### Git

First chart shows data directly from Dashboard. Second chart shows the same data grouped by projects provided by Mozilla, built specifically for this report.

In [99]:
s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
s.aggs.bucket('projects', 'terms', field='project', size=10)\
    .bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

result = s.execute()
            
df = stack_by_time(result=result, group_column='Project', time_column='Time', value_column='# Authors',\
        group_field='projects', time_field='time', value_field='contributors')

print_stacked_bar(df=df, time_column='Time', value_column='# Authors', group_column='Project')

**Git authors over time using Dashboard Data**

In [22]:
###
## GET DATA BY REPO FROM ES AND MERGE AGAINST PROJECTS SPREADSHEET
###

s = create_search(source='git')

# General filters
s = add_general_date_filters(s)
s = add_bot_filter(s)
s = add_merges_filter(s)

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gte': initial_date, 'lt': 'now/y'})
s.aggs.bucket('repos', 'terms', field='repo_name', size=100000)\
    .bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .bucket('contributors', 'terms', field='author_uuid', size=1000000)

result = s.execute()

# Process results to build a DataFrame
i = 0
repos_evo_df = pandas.DataFrame(columns=['Repo', 'Time', 'uuid'])
for repo in result.to_dict()['aggregations']['repos']['buckets']:
    for time in repo['time']['buckets']:
        for author in time['contributors']['buckets']:
            repos_evo_df.loc[len(repos_evo_df)] = [repo['key'], time['key_as_string'], author['key']]
            i += 1
            if i % 10000 == 0:
                print(i, end=', ')

merged_evo_df = repos_evo_df.merge(projects['Github'], on='Repo', how='left')

# Group By project
projects_evo_df = merged_evo_df.groupby(['Project', 'Time']).agg({'uuid': pandas.Series.nunique, 
                                                         'Repo': pandas.Series.nunique})
projects_evo_df = projects_evo_df.sort_values(by='uuid', ascending=0)    

# Plot it
print_stacked_bar(df=projects_evo_df.reset_index(), time_column='Time', value_column='uuid',
                  group_column='Project')

10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 

**Git authors over time using Spreadsheet Data**

#### Bugzilla

In [23]:
s = create_search(source='bugzilla')

s = s.filter('range', creation_ts={'gte': initial_date, 'lt': 'now/y'})

# Unique count of Authors by Project
s.aggs.bucket('product', 'terms', field='product', size=100000)\
    .bucket('component', 'terms', field='component', size=100000)\
    .bucket('time', 'date_histogram', field='creation_ts', interval='quarter')\
    .bucket('contributors', 'terms', field='author_uuid', size=100000)
result = s.execute()

print('q1')

# Process results to build a DataFrame
i = 0
repos_evo_df = pandas.DataFrame(columns=['Product', 'Component', 'Time', 'uuid'])
for product in result.to_dict()['aggregations']['product']['buckets']:
    for component in product['component']['buckets']:
        for time in component['time']['buckets']:
            for author in time['contributors']['buckets']:
                repos_evo_df.loc[len(repos_evo_df)] = [product['key'], component['key'], 
                                                       time['key_as_string'], author['key']]
                i += 1
                if i % 10000 == 0:
                    print(i, end=', ')

merged_evo_df = repos_evo_df.merge(projects['Bugzilla'], on=['Product', 'Component'], how='left')

# Group By project
projects_evo_df = merged_evo_df.groupby(['Project', 'Time']).agg({'uuid': pandas.Series.nunique,
                                                                  'Product': pandas.Series.nunique,
                                                                  'Component': pandas.Series.nunique})
projects_evo_df = projects_evo_df.sort_values(by='uuid', ascending=0)    

# Plot it
print_stacked_bar(df=projects_evo_df.reset_index(), time_column='Time', value_column='uuid',
                  group_column='Project')

q1
10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000, 170000, 180000, 190000, 200000, 210000, 220000, 230000, 240000, 250000, 260000, 270000, 280000, 290000, 300000, 310000, 

#### List of organizations


In [31]:
s = create_search(source='git')

s = add_bot_filter(s)
s = add_merges_filter(s)

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('organizations', 'terms', field='author_org_name', size=100)\
    .metric('commits', 'cardinality', field='hash', precision_threshold=100000)
result = s.execute()

In [32]:
print_df(result=result, group_field='organizations', value_field='commits', \
         group_column='Organization', value_column='# Commits')

,Organization,# Commits
0,Mozilla Staff,1364303
1,Community,679867
2,Mozilla Reps,481


### Contributors by organization
#### Git

In [33]:
s = create_search(source='git')

s = add_bot_filter(s)
s = add_merges_filter(s)

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('organizations', 'terms', field='author_org_name', size=100).\
    metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)
result = s.execute()

In [34]:
print_df(result=result, group_field='organizations', value_field='contributors', \
         group_column='Organization', value_column='# Contributors')

,Organization,# Contributors
0,Mozilla Staff,2182
1,Community,13043
2,Mozilla Reps,2


#### Bugzilla

In [10]:
s = create_search(source='bugzilla')

s = add_bot_filter(s)

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('organizations', 'terms', field='author_org_name', size=100).\
    metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)
result = s.execute()

In [11]:
print_df(result=result, group_field='organizations', value_field='contributors', \
         group_column='Organization', value_column='# Contributors')

,Organization,# Contributors
0,Community,181892
1,Mozilla Staff,2784
2,Mozilla Reps,3


### Number of contributors by organization over time
#### Git

In [57]:
s = create_search(source='git')

s = add_bot_filter(s)
s = add_merges_filter(s)
s = add_general_date_filters(s)

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gte': 'now/y-2y', 'lt': 'now/y'})
s.aggs.bucket('org', 'terms', field='author_org_name', size=10)\
    .bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

result = s.execute()

authors_org_df = stack_by_time(result=result, group_column='Organization', time_column='Time', value_column='# Contributors',\
        group_field='org', time_field='time', value_field='contributors')


In [58]:
print_stacked_bar(df=authors_org_df, time_column='Time', value_column='# Contributors', group_column='Organization')

### Contributors by groups: hired by Mozilla, the rest
#### Git

In [24]:
s = create_search(source='git')

s = add_bot_filter(s)
s = add_merges_filter(s)
s = add_general_date_filters(s)

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gte': initial_date, 'lt': 'now/y'})
s.aggs.bucket('org', 'terms', field='author_org_name', size=10)\
    .bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

result = s.execute()

authors_org_df = stack_by_cusum(result=result, group_column='Organization', time_column='Time',\
                                value_column='# Contributors', group_field='org', time_field='time',\
                                value_field='contributors', staff_org_names=['Mozilla Staff'],\
                                staff_org='Employees')

Mozilla Staff -> Employees
Community -> Non-Employees
Mozilla Reps -> Non-Employees


In [25]:
print_stacked_bar(df=authors_org_df, time_column='Time', value_column='# Contributors', group_column='Organization')

#### Bugzilla

In [26]:
s = create_search(source='bugzilla')

s = add_bot_filter(s)

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', creation_ts={'gte': initial_date, 'lt': 'now/y'})
s.aggs.bucket('org', 'terms', field='author_org_name', size=10)\
    .bucket('time', 'date_histogram', field='creation_ts', interval='quarter')\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

result = s.execute()

authors_org_df = stack_by_cusum(result=result, group_column='Organization', time_column='Time',\
                                value_column='# Contributors', group_field='org', time_field='time',\
                                value_field='contributors', staff_org_names=['Mozilla Staff'],\
                                staff_org='Employees')

Mozilla Staff -> Employees
Community -> Non-Employees
Mozilla Reps -> Non-Employees


In [27]:
print_stacked_bar(df=authors_org_df, time_column='Time', value_column='# Contributors', group_column='Organization')

#### Contributors by gender
**TODO**: Pending of running gender study over the data.

#### Number of contributors by gender over time
**TODO**: Pending of running gender study over the data.

#### Time of first and last commit for each contributor
**TODO** : provide plots similar to:
Attracted developers: https://analytics.mozilla.community:443/goto/1be7d078d3dda22bf2ed097d8e465fb9
Last commit developers: https://analytics.mozilla.community:443/goto/647ec69fc6b9d163827aa281ed3bee61

#### Length of period of activity for each contributor


#### Contributors by time zone (when possible)

https://analytics.mozilla.community:443/goto/9e930eefe59a7c90331d922887b2aee6

#### Contributors by city name (when possible)